## Práctica: implementación del mecanismo de auto-atención con enmascaramiento del modelo Transformer

Vamos a implementar el mecanismo de auto-atención con enmascaramiento del modelo Transformer en Pytorch. Para ello, vamos a seguir los pasos descritos anteriormente y suponer que ya tenemos las matrices de consultas (Q), claves (K) y valores (V) para cada token en la secuencia.

In [15]:
import torch
import math
from torch.nn.functional import scaled_dot_product_attention
from transformer_attention import TransformerAttention

### Paso 1: Cálculo de las puntuaciones de atención
Matrices de consultas (Q), claves (K) y valores (V):

In [16]:
Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

El enmascaramiento durante la etapa del decodificador en los modelos Transformer es crucial para evitar que el decodificador tenga acceso a información futura, especialmente en tareas de generación secuencial como la traducción automática o la generación de texto. Este concepto se conoce como "enmascaramiento de atención causal".

En el contexto de los Transformers, el decodificador genera una salida secuencialmente, palabra por palabra. Durante la generación de cada palabra, es importante que el modelo solo tenga en cuenta las palabras anteriores y no las futuras, ya que estas últimas no deberían estar disponibles (en un escenario de generación de texto, por ejemplo, las palabras futuras aún no se han generado).

Una vez realizado el resultado debe ser:

<table>
<tr>
<td><b>z1</b></td><td>1.0000</td><td>0.0000</td><td>0.0000</td>
</tr>
<tr>
<td><b>z2</b></td><td>0.4568</td><td>0.5432</td><td>0.0000</td>
</tr>
<tr>
<td><b>z3</b></td><td>0.3219</td><td>0.3332</td><td>0.3449</td>
</tr>
<tr>
<td><b>z4</b></td><td>0.2309</td><td>0.5130</td><td>0.5260</td>
</tr>
</table>

#### **Objetivos de la práctica**

- Entender con detalle el funcionamiento del mecanismo de auto-atención con enmascaramiento.
- Practicar las operaciones matriciales en PyTorch.

### Paso 2: Escalado de las puntuaciones de atención

In [28]:
scores = torch.matmul(Q, K.t())

### Paso 3: Dividir las puntuaciones de atención por la raíz cuadrada de la dimensión de las consultas

In [18]:
d_k = K.size(-1)
scaled_scores = scores / math.sqrt(d_k)

###  Paso 4: Aplicación de la matriz de enmascaramiento

In [19]:
mask = torch.triu(torch.ones(scaled_scores.size(-1), scaled_scores.size(-1)), diagonal=1)
mask = mask == 1
masked_scaled_scores = scaled_scores.masked_fill(mask, float('-inf'))

### Paso 5: Aplicación de la función Softmax

In [20]:
attention = torch.nn.functional.softmax(masked_scaled_scores, dim=1)

### Paso 6: Multiplicación con la matriz de valores (V)

In [24]:
attention_vectors = torch.matmul(attention, V)
attention_vectors = TransformerAttention.scaled_dot_product_attention(Q, K, V)
print(attention_vectors)

tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])


## Comparación con la implementación de Pytorch

In [25]:
z = scaled_dot_product_attention(Q, K, V, is_causal=True)

In [26]:
att = torch.round(attention_vectors * 10000) / 10000
z = torch.round(z * 10000) / 10000

print(att==z)	

tensor([[True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True]])
